In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                !pip install pyspark

In [ ]:
import os
import math
import datetime
import pyspark.sql.functions as sf
from pyspark.sql.functions import desc
from pyspark.sql.window import Window
from pyspark import SparkConf, SparkContext
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.sql.types import TimestampType
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import desc
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import IntegerType
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml import Pipeline
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import logging 

logging.getLogger("py4j").setLevel(logging.ERROR)

#Start the session and read the data
#-----------------------------------
spark = SparkSession.builder\
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('Recommendation_system').getOrCreate()


#Read and Show the data
#------------
df = spark.read \
    .options(header = False, inferSchema = False)  \
    .csv('/content/August_n_September_Processed.csv')

df = df.dropna()
df.show(10,truncate=True)
print("Shape of the raw read data : ", (df.count(), len(df.columns)))

+---+----+---+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+---+
|_c0| _c1|_c2|            _c3|                 _c4|                 _c5|                _c6|                 _c7|                 _c8|_c9|
+---+----+---+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+---+
|  1| 0.0| PV| 185.215.133.52|020-08-01 03:05:2...|&#214;ğrenme G&#2...|         Murat Bala|Nobel Akademik Ya...|      Eğitim / Diğer|  1|
|  2| 1.0| PV|   66.249.64.87|020-07-31 19:54:1...|Muhtasar&#39;ul K...|    Ahmed el-Kuduri|      Beka Yayınları|İslam / İslam Eği...|  7|
|  3| 2.0| PV|   88.241.13.62|020-08-01 03:05:5...|Sağlık Bilimleri ...|           Kolektif|    Alter Yayıncılık|Genel Konular / R...|  2|
|  4| 3.0| PV|   66.249.64.89|020-08-01 03:04:3...|Sek&#252;ler Topl...|Bryan Ronald Wilson|Pegem Akademi Yay...|       İslam / Diğer|  3|
|  5| 4.0| PV|  176.121.106

In [ ]:


df = df.withColumnRenamed("_c3", "user")
df = df.withColumnRenamed("_c5", "item")

df = df.groupBy("user", "item", "_c6", "_c7", "_c8", "_c9").count().orderBy(desc('count'))

df = df.withColumnRenamed("count", "rating")  
df.show(10, truncate=False)



+--------------+------------------------------------------------------------------+----------------------------+----------------------------+------------------------------------------+---+------+
|user          |item                                                              |_c6                         |_c7                         |_c8                                       |_c9|rating|
+--------------+------------------------------------------------------------------+----------------------------+----------------------------+------------------------------------------+---+------+
|31.206.145.97 |Hayel’e &#199;eyrek Kala                                          |Yeliz Yıldız                |BB Kitap                    |Edebiyat / Roman                          |4  |1629  |
|176.218.50.116|Hayel’e &#199;eyrek Kala                                          |Yeliz Yıldız                |BB Kitap                    |Edebiyat / Roman                          |4  |1278  |
|31.177.252.136|Otiz

In [ ]:
df = df.filter(df.rating > 5)
#row1 = df2.agg({"rating": "max"}).collect()[0]
#print(row1)

df.describe().show()

'''
Row(max(rating)=893)
+-------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+
|summary|          user|                item|       ProductPerson|        ProductBrand|     ProductCategory|        ProductPrice|           rating|
+-------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+
|  count|          1007|                1007|                1007|                1007|                1007|                1007|             1007|
|   mean|          null|                null|                null|                null|                null|   4.223340040241449|20.48361469712016|
| stddev|          null|                null|                null|                null|                null|    9.16781107079959|39.31373881083789|
|    min|109.36.143.120|&#199;&#246;z&#25...|#252;nyaya H&#252...|"""&#214;zdemir A...|&#199;ocuk Kitapları|""":""Edebiyat / ...|               11|
|    max| 95.70.130.102|              Şurina|     Şura G&#252;rel|      Şule Yayınları|   ı Kredi Yayınları|                   ı|              893|
+-------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+
'''

+-------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|summary|        user|                item|                 _c6|                 _c7|                 _c8|                 _c9|            rating|
+-------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|  count|        7962|                7962|                7962|                7962|                7962|                7962|              7962|
|   mean|        null|  1466.7142857142858|                null|                null|                null|   4.164750957854406| 9.702838482793268|
| stddev|        null|    979.439927422081|                null|                null|                null|  11.843453870131759|27.264421676276715|
|    min|10.234.0.102|% 100 D&#252;ş&#2...| H&#252;seyin Hoc...|"""""""&#214;zdem...|&#199;ocuk Kitapları|""""""":""""

'\nRow(max(rating)=893)\n+-------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+\n|summary|          user|                item|       ProductPerson|        ProductBrand|     ProductCategory|        ProductPrice|           rating|\n+-------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+\n|  count|          1007|                1007|                1007|                1007|                1007|                1007|             1007|\n|   mean|          null|                null|                null|                null|                null|   4.223340040241449|20.48361469712016|\n| stddev|          null|                null|                null|                null|                null|    9.16781107079959|39.31373881083789|\n|    min|109.36.143.120|&#199;&#246;z&#25...|#252;nyaya H&#252...|"""&#214;zdemir

In [ ]:
#String Indexer
#--------------

# Define a list of stages in your pipeline. The string indexer will be one stage
stages = []
categoricalColumns = [item[0] for item in df.dtypes if item[1].startswith('string')] #Stack all string type columns into
                                                                                     #categoricalColumns

for categoricalCol in categoricalColumns:
    #create a string indexer for those categorical values and assign a new name including the word 'Index'
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index', handleInvalid= "keep")

    #Append the string Indexer to our list of stages
    stages += [stringIndexer]

#Create the pipeline. Assign the stages list to the pipeline key word stages
pipeline = Pipeline(stages = stages)
#fit the pipeline to our dataframe
pipelineModel = pipeline.fit(df)
#transform the dataframe
df = pipelineModel.transform(df)
df.show(10, truncate=False)


+--------------+------------------------------------------------------------------+----------------------------+----------------------------+------------------------------------------+---+------+---------+---------+--------+--------+--------+--------+
|user          |item                                                              |_c6                         |_c7                         |_c8                                       |_c9|rating|userIndex|itemIndex|_c6Index|_c7Index|_c8Index|_c9Index|
+--------------+------------------------------------------------------------------+----------------------------+----------------------------+------------------------------------------+---+------+---------+---------+--------+--------+--------+--------+
|31.206.145.97 |Hayel’e &#199;eyrek Kala                                          |Yeliz Yıldız                |BB Kitap                    |Edebiyat / Roman                          |4  |1629  |1081.0   |26.0     |46.0    |66.0    |0.0     |3.

In [ ]:
(train, valid, test) = df.randomSplit([0.6, 0.2, 0.2], seed = 0)

#Total number of the users
print("Training :", train.count()," validation :", valid.count(), "test : ", test.count())
train.show(10, truncate=False)


Training : 4798  validation : 1575 test :  1589
+--------------+------------------------------------------------------------------+----------------------------+----------------------------+------------------------------------------+---+------+---------+---------+--------+--------+--------+--------+
|user          |item                                                              |_c6                         |_c7                         |_c8                                       |_c9|rating|userIndex|itemIndex|_c6Index|_c7Index|_c8Index|_c9Index|
+--------------+------------------------------------------------------------------+----------------------------+----------------------------+------------------------------------------+---+------+---------+---------+--------+--------+--------+--------+
|176.227.126.75|Kelime K&#246;ken                                                 |Mehmet Emin Katırcı         |Bilge K&#252;lt&#252;r Sanat|Edebiyat / Dil Bilim                      |2  |143   |2

In [ ]:

train = train.withColumnRenamed("item", "ProductName") 
train = train.withColumnRenamed("user", "IP") 





train = train.withColumnRenamed("userIndex", "user") 
train = train.withColumnRenamed("itemIndex", "item") 

In [ ]:
import logging 

logging.getLogger("py4j").setLevel(logging.ERROR)


#Model Training

ranks = [5]
reguls = [0.01]
iters = [5]
alpha = [10]

als = ALS(rank= 5, maxIter= 5, alpha = float(10), implicitPrefs=True, seed=0)

model = als.fit(train)




In [ ]:
test.show(5, truncate = False)

test = test.withColumnRenamed("item", "ProductName") 
test = test.withColumnRenamed("user", "IP") 




test.show(5, truncate = False)

test = test.withColumnRenamed("userIndex", "user") 
test = test.withColumnRenamed("itemIndex", "item") 

+-------------+-------------------------------------------------------+----------------------+-------------------------------------+------------------------------------------+---+------+---------+---------+--------+--------+--------+--------+
|user         |item                                                   |_c6                   |_c7                                  |_c8                                       |_c9|rating|userIndex|itemIndex|_c6Index|_c7Index|_c8Index|_c9Index|
+-------------+-------------------------------------------------------+----------------------+-------------------------------------+------------------------------------------+---+------+---------+---------+--------+--------+--------+--------+
|46.196.193.48|İki B&#252;y&#252;k D&#252;nya Sistemi Hakkında Diyalog|Galileo Galilei       |İş Bankası K&#252;lt&#252;r Yayınları|Genel Konular                             |4  |157   |32.0     |4753.0   |2291.0  |3.0     |10.0    |3.0     |
|46.196.92.87 |T&#252;rk Usu

In [ ]:
test.show(5, truncate = False)

+-------------+-------------------------------------------------------+----------------------+-------------------------------------+------------------------------------------+---+------+------+------+--------+--------+--------+--------+
|IP           |ProductName                                            |_c6                   |_c7                                  |_c8                                       |_c9|rating|user  |item  |_c6Index|_c7Index|_c8Index|_c9Index|
+-------------+-------------------------------------------------------+----------------------+-------------------------------------+------------------------------------------+---+------+------+------+--------+--------+--------+--------+
|46.196.193.48|İki B&#252;y&#252;k D&#252;nya Sistemi Hakkında Diyalog|Galileo Galilei       |İş Bankası K&#252;lt&#252;r Yayınları|Genel Konular                             |4  |157   |32.0  |4753.0|2291.0  |3.0     |10.0    |3.0     |
|46.196.92.87 |T&#252;rk Usul&#252; Londra          

In [ ]:
model.params

[Param(parent='ALS_264808cee815', name='blockSize', doc='block size for stacking input data in matrices. Data is stacked within partitions. If block size is more than remaining data in a partition then it is adjusted to the size of this data.'),
 Param(parent='ALS_264808cee815', name='coldStartStrategy', doc="strategy for dealing with unknown or new users/items at prediction time. This may be useful in cross-validation or production scenarios, for handling user/item ids the model has not seen in the training data. Supported values: 'nan', 'drop'."),
 Param(parent='ALS_264808cee815', name='itemCol', doc='column name for item ids. Ids must be within the integer value range.'),
 Param(parent='ALS_264808cee815', name='predictionCol', doc='prediction column name.'),
 Param(parent='ALS_264808cee815', name='userCol', doc='column name for user ids. Ids must be within the integer value range.')]

In [ ]:
model.predictionCol

Param(parent='ALS_264808cee815', name='predictionCol', doc='prediction column name.')

In [ ]:
model.userFactors.orderBy("id").collect()

[Row(id=0, features=[-1.2457048892974854, 1.1849135160446167, 0.000475430628284812, 2.1544055938720703, -2.7376320362091064]),
 Row(id=1, features=[-0.32365480065345764, 2.042970657348633, 0.02833973616361618, 2.839386463165283, 1.6626924276351929]),
 Row(id=2, features=[0.3933482766151428, 0.8323531150817871, 2.5507030487060547, 1.7630130052566528, -0.8063254952430725]),
 Row(id=3, features=[0.8180915117263794, 1.5569062232971191, 2.9773623943328857, 1.2638601064682007, 0.3073148727416992]),
 Row(id=4, features=[-3.705716609954834, 0.4379235804080963, 0.7087895274162292, 0.10975401848554611, -0.716830849647522]),
 Row(id=5, features=[-1.3355166912078857, -1.7291914224624634, -0.8300746083259583, 2.969939947128296, 0.9718936681747437]),
 Row(id=6, features=[-2.133401393890381, 1.702711582183838, -0.41571441292762756, 0.5732183456420898, 0.9100170731544495]),
 Row(id=7, features=[0.33763042092323303, 2.4592502117156982, -0.8615004420280457, -0.04606239125132561, -0.03625273331999779]),


In [ ]:
predictions = sorted(model.transform(test).collect(), key=lambda r: r[0])

In [ ]:
user_recs = model.recommendForAllUsers(15)

In [ ]:
user_recs.where(user_recs.user == 0).select("recommendations.item", "recommendations.rating").collect()

#item_recs = model.recommendForAllItems(3)
#item_recs.where(item_recs.item == 2).select("recommendations.user", "recommendations.rating").collect()


[Row(item=[38, 23, 71, 70, 50, 11, 13, 1046, 40, 6, 42, 284, 65, 59, 804], rating=[4.471241474151611, 4.219755172729492, 4.083179473876953, 4.071332931518555, 3.9779105186462402, 3.925535202026367, 3.8833751678466797, 3.739535093307495, 3.694912910461426, 3.496858835220337, 3.395853042602539, 3.3381125926971436, 2.8628766536712646, 2.836012840270996, 2.6654510498046875])]

In [ ]:
user_recs.where(user_recs.user == 0).select("recommendations.item", "recommendations.rating").collect()


[Row(item=[38, 23, 71, 70, 50, 11, 13, 1046, 40, 6, 42, 284, 65, 59, 804], rating=[4.471241474151611, 4.219755172729492, 4.083179473876953, 4.071332931518555, 3.9779105186462402, 3.925535202026367, 3.8833751678466797, 3.739535093307495, 3.694912910461426, 3.496858835220337, 3.395853042602539, 3.3381125926971436, 2.8628766536712646, 2.836012840270996, 2.6654510498046875])]

In [ ]:
first = user_recs.where(user_recs.user == 0).select("recommendations.item").collect()
var = 10
final = 0
while final < 6:
  for item in first:
    for i in range(0, var):
      #print('item:', item[0][i], '\n')
      asd = test.filter(test.item == item[0][i])
      print("asd.item:", len(asd.collect()))
      if  len(asd.collect()) == 0:
        
        var += 1
        continue      

      else:
        test.filter(test.item == item[0][i]).show()
        
      final += 1
      print("Final Val:", final)





asd.item: 3
+-------------+--------------------+--------------+------------+----------------+---+------+------+----+--------+--------+--------+--------+
|           IP|         ProductName|           _c6|         _c7|             _c8|_c9|rating|  user|item|_c6Index|_c7Index|_c8Index|_c9Index|
+-------------+--------------------+--------------+------------+----------------+---+------+------+----+--------+--------+--------+--------+
| 31.206.29.83|Jamaika’da Bir Fı...|Richard Hughes|Jaguar Kitap|Edebiyat / Roman|  3|     7|1092.0|38.0|    62.0|   165.0|     0.0|     2.0|
|   85.99.42.7|Jamaika’da Bir Fı...|Richard Hughes|Jaguar Kitap|Edebiyat / Roman|  3|     7|2107.0|38.0|    62.0|   165.0|     0.0|     2.0|
|46.196.196.39|Jamaika’da Bir Fı...|Richard Hughes|Jaguar Kitap|Edebiyat / Roman|  3|     6|  16.0|38.0|    62.0|   165.0|     0.0|     2.0|
+-------------+--------------------+--------------+------------+----------------+---+------+------+----+--------+--------+--------+--------+



In [ ]:

test.filter(test.user == 0).show()

In [ ]:

from pyspark.ml.evaluation import RegressionEvaluator

prd= model.transform(test)
prd.drop(prd.prediction == None)
prd.prediction.show(50)
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(prd)
print("RMSE="+str(rmse))

